<h1>SI 670 Kaggle Competition 4</h1>

Team GSIs

In [7]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('INFO')
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# import tensorflow as tf
import tensorflow_datasets as tfds
from keras.preprocessing.text import Tokenizer
from keras import Sequential
from keras import Model, Sequential, layers
from keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras import Input 

In [8]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

<h2>Load datasets</h2>

In [4]:
len(list(os.listdir("../data/images_train/bathroom"))), len(list(os.listdir("../data/images_train/kitchen")))

FileNotFoundError: [Errno 2] No such file or directory: '../data/images_train/bathroom'

In [9]:
data = tf.keras.utils.image_dataset_from_directory("../images_train", 
                                                   image_size = (256, 256), 
                                                   class_names = ['bathroom', 'kitchen'], 
                                                   shuffle = True,
                                                   batch_size = 32)

# run

Found 616 files belonging to 2 classes.


2023-11-22 19:32:13.755942: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-11-22 19:32:13.755974: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-22 19:32:13.755982: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-22 19:32:13.756017: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-22 19:32:13.756033: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
X, y = zip(*data)

X = np.concatenate(X)
y = np.concatenate(y)
# run

In [11]:
len(X), len(y)
# run

(616, 616)

In [53]:
img_width, img_height = 256, 256
batch_size = 8

In [51]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, shear_range=0.2, zoom_range=0.2)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [59]:
train_generator = train_datagen.flow_from_directory("../images_train", target_size=(img_width, img_height), batch_size=8, class_mode='binary')


Found 616 images belonging to 2 classes.


<h2>Preprocess Images</h2>

In [12]:
#flip labels so bathroom is 1
y = np.where(y == 1, 0, 1)
# run

In [55]:
X = np.apply_along_axis(lambda s: s / 255.0, axis = 1, arr = X)

<h2>Split</h2>

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size = 0.2)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
# run

In [30]:
train_datagen.fit(X_train)
# X_val = train_datagen.fit(X_val)
test_datagen.fit(X_test)

In [175]:
test_datagen.fit(X_test)

<h2>Train Model</h2>

In [ ]:
def skip_block(x):
    x_skip = x.copy()

    x = tf.keras.layers.Conv2D(64, (3, 3), strides = (2, 2), kernel_regularizer = l2(0.001))(x)
    x = tf.keras.layers.MaxPooling2D()

In [39]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (255-x)
  else:
    x
  return x
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invertx = random_invert()

In [8]:
IMG_SIZE = 256

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  # random_invert(),
])

# run

In [22]:
IMG_SIZE = 256

model = Sequential([
    # resize_and_rescale,
    # data_augmentation,
    #layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1./255),
    #layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomContrast(0.2),
    layers.RandomZoom(0.2),

    layers.Conv2D(128, (3,3), activation='relu',  kernel_regularizer = l2(0.0001)),
    layers.MaxPooling2D(),
    layers.Conv2D(256,(3,3), activation='relu',  kernel_regularizer = l2(0.0001)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3),  activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, (3,3),  activation='relu'),
    layers.MaxPooling2D(3,3),
    layers.Flatten(),
    layers.Dense(64),
    layers.Dense(1, activation='sigmoid'),
    # input
    # tf.keras.layers.Conv2D(128, (3, 3), input_shape = (256, 256, 3)), 
    # # tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5),  
    # # tf.keras.layers.BatchNormalization(axis = -1),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'), 

    
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # # tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5, input_shape = (256, 256, 3)), 
    # # tf.keras.layers.Conv2D(128, (3, 3), kernel_regularizer = l2(0.0001), activation = 'relu'), 
    # tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5, input_shape = (256, 256, 3)), 
    # tf.keras.layers.Conv2D(256, (3, 3), activation = 'relu'), 

    
    # # tf.keras.layers.Conv2D(256, (3, 3), activation = 'relu', input_shape = (256, 256, 3)), 
    # # tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'), 
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'), 
    # # tf.keras.layers.Dropout(0.1),
    # # tf.keras.layers.Conv2D(86, (3, 3), activation = 'relu'),
    # # tf.keras.layers.MaxPooling2D((3, 3)),
    # tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'), 
    # # tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'), 
    # tf.keras.layers.Dropout(0.01),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Flatten(), 
    # tf.keras.layers.Dense(50), 
    # tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [6]:
model = Sequential([
    # input
    resize_and_rescale,
    data_augmentation,
    # tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    # tf.keras.layers.MaxPooling2D((3, 3)),
    # tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5, input_shape = (256, 256, 3)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # tf.keras.layers.MaxPooling2D((3, 3)),   
    
    tf.keras.layers.Conv2D(32, (3, 3), kernel_regularizer = l2(0.0001), input_shape = (256, 256, 3)), 
    tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5),  
    
    # tf.keras.layers.Conv2D(64, (3, 3), kernel_regularizer = l2(0.0001), activation = 'relu'), 

    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'), 

    
    tf.keras.layers.MaxPooling2D((3, 3)),
    # tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5, input_shape = (256, 256, 3)), 
    # tf.keras.layers.Conv2D(128, (3, 3), kernel_regularizer = l2(0.0001), activation = 'relu'), 
    # tf.keras.layers.BatchNormalization(axis = -1, epsilon = 2e-5, input_shape = (256, 256, 3)), 
    # tf.keras.layers.Conv2D(128, (3, 3), kernel_regularizer = l2(0.0001), activation = 'relu'), 

    
    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu', input_shape = (256, 256, 3)), 
    tf.keras.layers.Conv2D(256, (3, 3), activation = 'relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'), 
    # tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu'), 
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(64), 
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

# run

NameError: name 'Sequential' is not defined

In [23]:
loss = BinaryCrossentropy()
model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy'])
# keras adam optimizer
# tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)


# run

In [60]:
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0.01, patience = 3, restore_best_weights = True)
model.fit(
    train_generator,
    epochs=10)
    # validation_data=validation_generator,
    # validation_steps=800)

Epoch 1/10
77/77 [==============================] - 52s 670ms/step - loss: 0.6716 - accuracy: 0.5974
Epoch 2/10
77/77 [==============================] - 52s 679ms/step - loss: 0.5461 - accuracy: 0.7029
Epoch 3/10
77/77 [==============================] - 52s 675ms/step - loss: 0.4193 - accuracy: 0.8214
Epoch 4/10
77/77 [==============================] - 52s 670ms/step - loss: 0.3748 - accuracy: 0.8425
Epoch 5/10
77/77 [==============================] - 52s 674ms/step - loss: 0.3512 - accuracy: 0.8474
Epoch 6/10
77/77 [==============================] - 52s 670ms/step - loss: 0.3366 - accuracy: 0.8815
Epoch 7/10
77/77 [==============================] - 52s 671ms/step - loss: 0.3618 - accuracy: 0.8442
Epoch 8/10
60/77 [======================>.......] - ETA: 11s - loss: 0.3665 - accuracy: 0.8292

KeyboardInterrupt: 

In [69]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import tensorflow as tf

In [24]:
early_stopping = EarlyStopping(monitor = 'accuracy', min_delta = 0.001, patience = 3, restore_best_weights = True)
# didnt do validation, just train and test
# model_history = model.fit(train_datagen.flow(X_train, y_train, batch_size=32,
#          subset='training'), epochs = 15, 
#                         #   callbacks = [early_stopping], 
#                           validation_data = train_datagen.flow(X_train, y_train, batch_size=8, subset="validation") )

model_history = model.fit(X_train, y_train, epochs = 15, batch_size = 8, callbacks = [early_stopping])

                          # validation_data = (X_val, y_val))


# run

Epoch 1/15


2023-11-22 19:35:25.425805: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


 2/62 [..............................] - ETA: 8s - loss: 1.7683 - accuracy: 0.5000  

2023-11-22 19:35:25.894076: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.
2023-11-22 19:35:26.042636: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


 4/62 [>.............................] - ETA: 8s - loss: 1.2142 - accuracy: 0.5312

...

27/62 [============>.................] - ETA: 5s - loss: 0.3637 - accuracy: 0.8565

2023-11-22 19:36:14.062767: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.
2023-11-22 19:36:14.205084: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


29/62 [=============>................] - ETA: 4s - loss: 0.3588 - accuracy: 0.8578

2023-11-22 19:36:14.346659: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.
2023-11-22 19:36:14.487861: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


KeyboardInterrupt: 

In [20]:
model_history = model.fit(X, y, epochs = 8, batch_size = 8 )

Epoch 1/8


2023-11-22 19:34:43.941510: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


 2/77 [..............................] - ETA: 14s - loss: 0.3324 - accuracy: 0.8750

2023-11-22 19:34:44.407348: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.
2023-11-22 19:34:44.599669: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


 4/77 [>.............................] - ETA: 11s - loss: 0.3070 - accuracy: 0.8750

...

58/77 [=====================>........] - ETA: 2s - loss: 0.3294 - accuracy: 0.8815

2023-11-22 19:34:52.695440: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.
2023-11-22 19:34:52.839682: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


59/77 [=====================>........] - ETA: 2s - loss: 0.3281 - accuracy: 0.8835

2023-11-22 19:34:52.981294: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


KeyboardInterrupt: 

In [371]:
model.save("../saved_weights/haley_submit_3")

INFO:tensorflow:Assets written to: ../saved_weights/haley_submit_3/assets


INFO:tensorflow:Assets written to: ../saved_weights/haley_submit_3/assets


<h2>Test Model</h2>

In [25]:
model.evaluate(X_test, y_test)
# model.evaluate(test_datagen.flow(X_test, y_test, batch_size = 8))

# run

4/4 [==============================] - 1s 265ms/step - loss: 0.3060 - accuracy: 0.9032


[0.30603399872779846, 0.9032257795333862]

In [45]:
#model.evaluate(X_val, y_val)
model.evaluate(train_datagen.flow(X_train, y_train, batch_size = 8, subset = 'validation'))

NameError: name 'train_datagen' is not defined

In [66]:
# y_pred_prob = model.predict(X_test)


y_pred_prob = model.predict(test_datagen.flow(X_test, y_test, batch_size = 8))

# run

16/16 [==============================] - 1s 29ms/step


In [67]:
y_pred_class = np.where(y_pred_prob > 0.5, 1, 0).flatten()

# run

In [68]:
matrix = confusion_matrix(y_pred_class, y_test)

# run

In [69]:
normalized_confusion_matrix = matrix / matrix.astype(float).sum(axis = 1)[:, np.newaxis]

# run

In [70]:
f = sns.heatmap(normalized_confusion_matrix, annot = True)
f.set(title = 'Normalized Confusion Matrix', xlabel = 'True', ylabel = 'Predicted')
plt.savefig("../plots/haley_heatmap_submission3.png")

# run

<h2>Make Prediction</h2>

In [106]:
pred_data = tf.keras.utils.image_dataset_from_directory("../images_test", image_size = (256, 256), labels = None)
pred_data = pred_data.unbatch()
pred_images = [img / 255.0 for img in pred_data]

Found 149 files belonging to 1 classes.


In [31]:
imgs = []
ids = []

for f in os.listdir("../images_test"):
    img = tf.keras.utils.load_img(os.path.join("../images_test", f), target_size = (256, 256, 3))
    # img = tf.keras.utils.img_to_array(img) / 255.0
    id = f.split(".")[0]
    ids.append(id)
    imgs.append(img)

# run

In [32]:
predictions = model.predict(np.array(np.array(imgs)))

# run


5/5 [==============================] - 0s 50ms/step


In [33]:
pred_data = tf.keras.utils.image_dataset_from_directory("../images_test", image_size = (256, 256), labels = None)
#pred_data = pred_data.unbatch()

Found 149 files belonging to 1 classes.


In [22]:
pred_data

<_PrefetchDataset element_spec=TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None)>

In [23]:
predictions = model.predict(pred_data)

5/5 [==============================] - 3s 178ms/step


<h2>Format predictions for submission</h2>

In [34]:
class_predictions = np.where(predictions > 0.5, 1, 0).flatten()

# run

In [35]:
df = pd.DataFrame({'id': ids, 'class': class_predictions})
# run

In [36]:
df.to_csv("../submit_12.csv", index = None)
# run

In [37]:
df

id  class
0    162      0
1    228      0
2    214      0
3    200      0
4    201      1
..   ...    ...
144  219      1
145  225      0
146  231      1
147  153      0
148  147      0

[149 rows x 2 columns]

In [453]:
w = np.matrix()

TypeError: matrix.__new__() missing 1 required positional argument: 'data'

In [456]:
w = np.array([])

In [489]:
w = np.empty((0, 0))

In [491]:
t = np.append(w, [[(0, 1)], [(1, 2)]])

In [494]:
b = [[1, 2], [2, 1]]

In [501]:
np.empty((0, 0))

array([], shape=(0, 0), dtype=float64)

In [511]:
a = np.array([[None, None], [None, None]])

In [507]:
a[0] = 2

In [512]:
a.shape

(2, 2)